In [1]:
import pickle
import numpy as np

# Charger les objets
with open("../models/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("../models/label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

X_train = np.load("../models/X_train.npy")
X_test = np.load("../models/X_test.npy")
y_train = np.load("../models/y_train.npy")
y_test = np.load("../models/y_test.npy")


Premier modèle : Régression logistique 

In [2]:
from sklearn.linear_model import LogisticRegression

#Entrainement du modèle

model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

#On mets max_iter = 500 pour être sur que l'algo ait assez de temps
# pour converger.


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


In [3]:
#Prédictions 

y_pred = model.predict(X_test)

In [5]:
#Evalutation du modèle : Matrice de confusion et accuracy 

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)

acc = accuracy_score(y_test, y_pred)
print("Accuracy :", acc)



[[ 402    0   67    1  161    0    0   80  200    0    0   86]
 [  31   14    1    1  288    0    0  465  397    8    0  394]
 [ 148    0  134    4   21    0    1   90  137    0    0   61]
 [   5    2    3   54   39    0    0  243  326    0    0  125]
 [  48    1    4    0 2833    0    0  481  368    5    0  654]
 [   0    0    0    0   86    0    0   47   51    2    0   14]
 [   6    0    1   35   25    0    0  126  150    0    0   57]
 [  92    8   69    6  849    0    0 1850  810    6    0  504]
 [ 126   12   27   10  684    0    3  948 1547    4    0  637]
 [   1    1    0    2  259    0    0  228  146    9    0  154]
 [   1    5    0    7  188    0    0  366  232    9    0  191]
 [  83    3   16   32  640    0    0  537  466    2    2 2015]]
Accuracy : 0.38902064119455426


On a 12 classes différentes ce qui nous donne un taux de 8% de classer correctement une musique si on le faisait aléatoirement. 
Ici notre accuracy est de 39% ce qui est largement supérieure au 8% aléatoire. 

Première conclusion : notre modèle a capturé une vraie strucutre dans les données, pas juste du bruit. Il lui reste cependant une bonne marge de progression !

Que nous dit la matrice de confusion ? 
Il prédit très bien les classes 4,0 et 2 avec respectivement 2833, 402 et 134 prédictions correctes.
Il surprédit les classes 4,7 et 8 (signe que le dataset est déséquilibré).
Cependant, il passe complétement à côté des classes 5,6 et 10 pour lesquelles il n'obtient aucune bonnes prédictions... 
(on peut calculer le recall pour s'en convaincre)

Explications possibles : Il confond des classes entre elles, le modèle n'arrive pas à trouver une frontière nette pour certaines classes. On atteint surement les limites de la régression logistique qui est trop linéaire pour un modèle de classsification à 12 classes.

Solutions : 
Passer à un MLP classifier qui permet de traiter un modèle non-linéaire.
vérifier le déséquilibre des classes
Regrouper les classes trop rares.
 





Quelles pistes si on souhaite maximiser ce modèle ? 

- Equilibrer le poids des classes (pour forcer le modèle à ne pas ignorer les classes minoritaires)
- Changer de solver ? 
- Régularisation : contrôle de C 
- Feature engineering 
- GridSearchCV (pour trouver directement les meilleurs hyperparamètres)
- Réduire le déséquilibre d'exemple entre les classes (SMOTE)
- Tester la version polynomiale 

## Méthodes ensemblistes : Random Forest puis XGBoost

2ème modèle : Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# === Modèle Random Forest ===

rf = RandomForestClassifier(
    n_estimators=300,       # nombre d’arbres (300 = bon compromis)
    max_depth=None,        # profondeur libre (laisser l'algo décider)
    min_samples_split=2,   # paramètres par défaut
    min_samples_leaf=1,
    random_state=8,
    n_jobs=-1              # utilise tous les cœurs du CPU
)

rf.fit(X_train, y_train)

# Prédictions
y_pred_rf = rf.predict(X_test)

# Accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy Random Forest :", acc_rf)

# Matrice de confusion
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(cm_rf)

# Rapport détaillé (Precision / Recall / F1)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))


Accuracy Random Forest : 0.5660079051383399
[[ 571    1   57    5   77    0    0   46  188    0   16   36]
 [  29  649   12   14  150   10   32  295  213   42    4  149]
 [  83   14  257    3   11    0    0   67   99    0    2   60]
 [   5   16    3  343   36    0    1  126  187    1    4   75]
 [  40   43    5    9 3115   20    7  306  325  124   75  325]
 [   0   10    0    0   28   30    0   50   65    6    1   10]
 [  11   48    4    5   18    0  129   45   52    4   18   66]
 [  63   79   58   30  336    6    7 2790  357  106  124  238]
 [ 105   91   37   53  454   29    9  420 2322    2   32  444]
 [   1   40    0    2  187    9    0  361   29   74    3   94]
 [  12   20    3    5  135    2   15  398   86    3  189  131]
 [  60   86   25   46  232    2   20  364  451   31   60 2419]]

=== Classification Report ===
               precision    recall  f1-score   support

ambient_chill       0.58      0.57      0.58       997
    asian_pop       0.59      0.41      0.48      1599
  

57 % de précision, pas mal, et mais autour de 55 % de f1-score pour chaque catégorie, sauf hiphop qui est à 19 %, sûrement car c'est une classe minoritaire ie support très faible (200), on va ajouter class_weight="balanced" pour voir si ça améliore

In [10]:
# === Modèle Random Forest ===

rf = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    random_state=8,
    n_jobs=-1
)

rf.fit(X_train, y_train)

# Prédictions
y_pred_rf = rf.predict(X_test)

# Accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy Random Forest :", acc_rf)

# Matrice de confusion
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(cm_rf)

# Rapport détaillé (Precision / Recall / F1)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))


Accuracy Random Forest : 0.5497145366710584
[[ 572    0   52    6   74    0    0   57  185    0   16   35]
 [  27  656   11   16  133   12   48  297  204   46    7  142]
 [  82   13  286    1   10    0    3   64   92    0    2   43]
 [   5   12    3  375   25    2   13  129  167    0    6   60]
 [  45   42    3    9 3028   33   16  315  332  149  103  319]
 [   0    9    0    1   19   59    0   20   66   13    4    9]
 [  11   39    3    9   18    0  178   46   41    2    9   44]
 [  66   76   60   37  336   65   22 2494  350  309  160  219]
 [ 112  104   39  120  414   37   26  425 2235   17   45  424]
 [   2   33    0    2  168   34    0  315   31  119    5   91]
 [  12   24    4    5  124    4   32  350   85   11  232  116]
 [  68   90   32   50  229    2   78  381  440   55   88 2283]]

=== Classification Report ===
               precision    recall  f1-score   support

ambient_chill       0.57      0.57      0.57       997
    asian_pop       0.60      0.41      0.49      1599
  

ça a un peu amélioré hipop mais pas le reste donc on est pas encore gagnant, on peut tenter de mettre + d'arbres et de limiter la profondeur

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# === Modèle Random Forest ===

rf = RandomForestClassifier(
    n_estimators=800,
    max_depth=25,
    class_weight="balanced",
    random_state=8,
    n_jobs=-1
)


rf.fit(X_train, y_train)

# Prédictions
y_pred_rf = rf.predict(X_test)

# Accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy Random Forest :", acc_rf)

# Matrice de confusion
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(cm_rf)

# Rapport détaillé (Precision / Recall / F1)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))

Accuracy Random Forest : 0.5497145366710584
[[ 570    2   54    7   79    0    0   53  186    0   15   31]
 [  30  691   10   19  127   12   48  266  198   48    8  142]
 [  80   17  287    2   13    0    2   58   91    0    2   44]
 [   5   17    2  379   26    2   13  124  169    1    5   54]
 [  45   46    2   10 3016   34   16  319  330  154  104  318]
 [   0   11    0    1   19   62    0   22   63   10    4    8]
 [  11   38    3   10   16    0  178   47   38    4    9   46]
 [  63   98   61   42  328   65   21 2443  363  323  172  215]
 [ 106  118   42  122  424   38   26  405 2231   17   45  424]
 [   2   36    0    3  166   34    0  307   29  125    5   93]
 [  13   22    4    5  124    4   32  347   83   12  242  111]
 [  70   97   33   61  221    3   79  357  432   58   92 2293]]

=== Classification Report ===
               precision    recall  f1-score   support

ambient_chill       0.57      0.57      0.57       997
    asian_pop       0.58      0.43      0.49      1599
  

ça change pas grand chose, on va recommencer au propre en améliorant le modème avec un RandomizedSearchCV pour trouver les meilleurs hyperparamètres qui optimisent le F1-score (pas l'accuracy comme il y a déséquilibre des classes)

In [11]:
from sklearn.model_selection import RandomizedSearchCV

rf_base = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": [300, 500, 800],
    "max_depth": [None, 15, 25, 35],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.3, 0.5]
}

search = RandomizedSearchCV(
    rf_base,
    param_distributions=param_dist,
    n_iter=25,              # nombre de combos testés
    cv=3,                   # 3-fold CV
    scoring="f1_macro",     # on optimise le F1-macro
    verbose=2,
    n_jobs=-1
)

search.fit(X_train, y_train)

print("Meilleurs hyperparamètres :", search.best_params_)
best_rf = search.best_estimator_


Fitting 3 folds for each of 25 candidates, totalling 75 fits
Meilleurs hyperparamètres : {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.5, 'max_depth': 25}


on a obtenu les meilleurs hyperparamètres, mtn on évalue ce meilleur modèle

In [12]:
from sklearn.metrics import accuracy_score, classification_report

y_pred_best = best_rf.predict(X_test)

print("Accuracy (RF optimisé) :", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best, target_names=le.classes_))


Accuracy (RF optimisé) : 0.5680720245937637
               precision    recall  f1-score   support

ambient_chill       0.58      0.56      0.57       997
    asian_pop       0.58      0.43      0.49      1599
    classical       0.56      0.44      0.49       596
 country_folk       0.66      0.43      0.52       797
   electronic       0.65      0.70      0.68      4394
       hiphop       0.28      0.17      0.21       200
   jazz_blues       0.58      0.33      0.42       400
  latin_world       0.54      0.66      0.59      4194
          pop       0.53      0.58      0.56      3998
   reggae_ska       0.20      0.10      0.14       800
rnb_soul_funk       0.38      0.21      0.27       999
   rock_metal       0.60      0.64      0.62      3796

     accuracy                           0.57     22770
    macro avg       0.51      0.44      0.46     22770
 weighted avg       0.56      0.57      0.56     22770



ça reste pas terrible, on va changer de modèle, on va essayer XGBost

In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.8,
    objective="multi:softmax",
    num_class=len(le.classes_),
    eval_metric="mlogloss",
    tree_method="hist",      # super rapide sur CPU
    random_state=42
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

print("Accuracy XGBoost :", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb, target_names=le.classes_))


Accuracy XGBoost : 0.5779973649538866
               precision    recall  f1-score   support

ambient_chill       0.60      0.55      0.57       997
    asian_pop       0.56      0.41      0.47      1599
    classical       0.57      0.46      0.51       596
 country_folk       0.65      0.44      0.53       797
   electronic       0.68      0.72      0.70      4394
       hiphop       0.29      0.20      0.23       200
   jazz_blues       0.57      0.36      0.44       400
  latin_world       0.55      0.67      0.60      4194
          pop       0.53      0.59      0.56      3998
   reggae_ska       0.30      0.16      0.21       800
rnb_soul_funk       0.46      0.25      0.32       999
   rock_metal       0.61      0.64      0.62      3796

     accuracy                           0.58     22770
    macro avg       0.53      0.45      0.48     22770
 weighted avg       0.57      0.58      0.57     22770



XGboost n'améliore pas tellement, mais on n'a pas réglé les hyperparamètres optimaux : 

In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

xgb_base = XGBClassifier(
    objective="multi:softprob",
    num_class=len(le.classes_),
    eval_metric="mlogloss",
    tree_method="hist",
    n_jobs=-1,
    random_state=42
)

# Petite grille, volontairement restreinte
param_dist = {
    "n_estimators": [300, 500, 700],
    "learning_rate": [0.03, 0.05, 0.08],
    "max_depth": [5, 7, 9],
    "min_child_weight": [1, 3],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.7, 1.0],
}

search_xgb_light = RandomizedSearchCV(
    xgb_base,
    param_distributions=param_dist,
    n_iter=8,          # très peu de combos
    cv=2,              # 2 folds seulement
    scoring="f1_macro",
    verbose=1,
    n_jobs=-1
)

search_xgb_light.fit(X_train, y_train)

print("Best params (light) :", search_xgb_light.best_params_)
best_xgb = search_xgb_light.best_estimator_

# Évaluation sur le test
y_pred_best = best_xgb.predict(X_test)

print("Accuracy XGB (light search) :", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best, target_names=le.classes_))


Fitting 2 folds for each of 8 candidates, totalling 16 fits
Best params (light) : {'subsample': 0.8, 'n_estimators': 700, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
Accuracy XGB (light search) : 0.5759771629336847
               precision    recall  f1-score   support

ambient_chill       0.60      0.55      0.58       997
    asian_pop       0.55      0.41      0.47      1599
    classical       0.55      0.47      0.51       596
 country_folk       0.67      0.45      0.54       797
   electronic       0.67      0.71      0.69      4394
       hiphop       0.29      0.20      0.23       200
   jazz_blues       0.58      0.36      0.45       400
  latin_world       0.55      0.67      0.60      4194
          pop       0.53      0.59      0.56      3998
   reggae_ska       0.31      0.17      0.22       800
rnb_soul_funk       0.48      0.23      0.31       999
   rock_metal       0.60      0.64      0.62      3796

     accuracy            

## Méthode basée sur un réseau de neurones

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128),  # 2 couches cachées
    activation='relu',
    solver='adam',
    batch_size=256,
    learning_rate_init=0.001,
    max_iter=80,          # nb d'epochs max
    early_stopping=True,  # s'arrête tout seul si ça n'améliore plus
    n_iter_no_change=5,
    random_state=8,
    verbose=True          # affiche la loss pendant l'entraînement
)

mlp.fit(X_train, y_train)

y_pred_mlp = mlp.predict(X_test)

print("Accuracy MLP :", accuracy_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp, target_names=le.classes_))


Iteration 1, loss = 1.71250178
Validation score: 0.450373
Iteration 2, loss = 1.57870537
Validation score: 0.470246
Iteration 3, loss = 1.52881678
Validation score: 0.483970
Iteration 4, loss = 1.49354308
Validation score: 0.491326
Iteration 5, loss = 1.46760144
Validation score: 0.495279
Iteration 6, loss = 1.44955476
Validation score: 0.504062
Iteration 7, loss = 1.43343791
Validation score: 0.507137
Iteration 8, loss = 1.41915169
Validation score: 0.509113
Iteration 9, loss = 1.40951542
Validation score: 0.510650
Iteration 10, loss = 1.39804704
Validation score: 0.513724
Iteration 11, loss = 1.38757061
Validation score: 0.518884
Iteration 12, loss = 1.38077619
Validation score: 0.515481
Iteration 13, loss = 1.37212781
Validation score: 0.521080
Iteration 14, loss = 1.36531422
Validation score: 0.516140
Iteration 15, loss = 1.35846034
Validation score: 0.518665
Iteration 16, loss = 1.35164166
Validation score: 0.523276
Iteration 17, loss = 1.34447849
Validation score: 0.519433
Iterat